# Collections

> Tuples and lists, sets and dictionaries, and L

We use fastai's test library. Our tests always preceed the implementation(s). They should silently go through, returning only the execution time.

In [ ]:
# uncomment this to install nbdev
# !pip install nbdev

from fastcore.test import test_eq
from timeit import timeit


## Tuples and Lists

Iterables produce Iterators via `iter()` such as all Collections.
Iterator yield the next element via `next()`. By convention, all Iterators
are also Iterables; the standard implementation being to return self.
Some examples:

* `(0, 1, 2, 3)` is a tuple, no iterator
* `range(4)` is a tuple, no iterator
* `naturals()` is an iterator
* `(1 / k for k in naturals())` is an iterator

## Histograms

We write a function `histogram(xs: list) -> dict` which returns a dictionary containing the
number of occurrences of each x in xs. Here is what `histogram` is supposed to do:

In [ ]:
def test_histogram(htg):
    def aux():
        xss = [[], [1], [0, 1, 2, 2, 3, 3, 3], list(range(500)) + list(range(500))]
        for xs in xss:
            h = htg(xs)
            for x in xs:
                test_eq(h[x], xs.count(x))
    return aux

Idea: Start with an empty dictionary; iterate over xs and count the occurrence of each x.

In [ ]:
#collapse
def histogram1(xs: list) -> dict:
    """
    :param xs: a list
    :return: histogram of xs: a dictionary indicating how often each x occurs in xs
    """
    result = {}
    for x in xs:
        result[x] = 1 if x not in result.keys() else result[x] + 1
    return result

Another idea: define the dictionary directly

In [ ]:
#collapse
def histogram2(xs: list) -> dict:
    """
    :param xs: a list
    :return: histogram of xs: a dictionary indicating how often each x occurs in xs
    """
    return dict([(x, xs.count(x)) for x in xs])

A little tweak: iterate over `set(xs)` rather than `xs`

In [ ]:
#collapse
def histogram3(xs: list) -> dict:
    """
    :param xs: a list
    :return: histogram of xs: a dictionary indicating how often each x occurs in xs
    """
    return dict([(x, xs.count(x)) for x in set(xs)])

Compare the results

In [ ]:
for htg in (histogram1, histogram2, histogram2 ):
    print(f"{htg.__name__ :<7} | {timeit(test_histogram(htg), number=100)}")

histogram1 | 2.0417352
histogram2 | 2.8792690000000003
histogram2 | 2.9637431000000003


### Indices

We write a function `index(book: list, keywords: list) -> dict` which returns the index of a book.
The `book` is given as a list of pages, each page being a list of words. `keywords` is the list of words to be indexed.
The result is a dictionary which contains for each keyword the set of pages where it occurs.
Here is what `index` is supposed to do:

In [ ]:
def test_index(idx):
    def aux():
        book = [['xx', 'yy', 'zz'],
                ['xx', 'uu', 'zz'],
                ['xx', 'yy', 'vv', 'tt']]
        keywords = ['xx', 'tt']

        index = idx(book, keywords)
        test_eq(keywords, list(index.keys()))
        for w, ps in index.items():
            for p in ps:
                test_eq(w in book[p], True)
    return aux

Idea: Start with an empty dictionary; iterate over all pages

In [ ]:
#collapse
def index(book: list, keywords: list) -> dict:
    """
    book[i]  = set of all words on page i
    keywords = set of all indexable words
    result[word] = list of pages containing word
    standard solution
    """
    result = {}
    keywords = set(keywords)
    for i, page in enumerate(book):
        for word in set(page) & keywords:
            if word not in result.keys():
                result[word] = []
            else:
                result[word].append(i)
    return result

A variant: replace each page with the intersection of itself and the keywords

In [ ]:
#collapse
def index1(book: list, keywords: list) -> dict:
    """
    book[i]  = set of all words on page i
    keywords = set of all indexable words
    result[word] = list of pages containing word
    """
    keywords = set(keywords)
    book = [set(page) & keywords for page in book]
    result = {}
    for i, page in enumerate(book):
        for word in page:
            if word not in result.keys():
                result[word] = []
            else:
                result[word].append(i)
    return result

Compare the results

In [ ]:
for idx in (index, index1):
    print(f"{idx.__name__ :<7} | {timeit(test_index(idx), number=1000)}")


index   | 0.036819500000000005
index1  | 0.03789010000000026


## Merging

We write a function `merge(xs, ys: list) -> list` which returns the merge of non-descending lists xs and ys.
Here is what `merge` is supposed to do.

In [ ]:
def test_merge(mrg):
    def aux():
        xs = [9, 11]
        ys = [2, 4, 5]
        test_eq(mrg(xs, ys), [2, 4, 5, 9, 11])

        for i in range(100):
            xs = i * [1]
            ys = 2 * i * [1]
            m = mrg(xs, ys)
            test_eq(3 * i * [1], m)
    return aux

We start with a non-recursive solution. Start with an empty result.
Let x and y be the next elements of xs and ys and assume x <= y.
Then x is appended to the result and replaced with the next element of xs.
Th algorithm stops if at least one of the lists is exhausted.

In [ ]:
#collapse
def merge(xs, ys: list) -> list:
    """
    :param xs: a non-descending list
    :param ys: a non-descending list
    :return: merge of xs and ys
    Standard solution. A bit tricky.
    """
    result = []
    # invariants:
    # x, y first elements of xs, ys, None if there is no first element
    x = xs.pop(0) if xs else None
    y = ys.pop(0) if ys else None

    while x and y:  # same as x is not None and y is not None
        if x <= y:  # get next element of xs for next loop
            result.append(x)
            x = xs.pop(0) if xs else None
        else:  # get next element of ys for next loop
            result.append(y)
            y = ys.pop(0) if ys else None

    # x or y may be left behind (but not both)
    if x:
        result.append(x)
    if y:
        result.append(y)

    # one of the remaining xs, ys is empty
    # the one which is not (if any) is appended to result
    if xs:  # same as len(xs) > 0
        result += xs
    if ys:
        result += ys

    return result

The recursive solution is simpler and slightly faster. The stack depth however equals the combined length of xs and ys.
Idea: Return xs if ys is empty and vice versa. If both lists are not empty, the smaller of xs[0] and ys[0]
is going to be the first element of the merged list to which you append the rest.

In [ ]:
#collapse
def merge1(xs, ys: list):
    """
    :param xs: a non-descending list
    :param ys: a non-descending list
    :return: merge of xs and ys
    This the definition of merge, and it does run!
    """
    if not xs:
        return list(ys)
    elif not ys:
        return list(xs)
    elif xs[0] <= ys[0]:
        return xs[:1] + merge1(xs[1:], ys)
    else:
        return ys[:1] + merge1(xs, ys[1:])

In [ ]:
for mrg in (merge, merge1):
    print(f"{mrg.__name__ :<6}| {timeit(test_merge(mrg), number=1)}")

merge | 0.1436288000000001
merge1| 0.14114170000000037


## Towers of Hanoi

We write a function `hanoi(n: int) -> list` which returns the list of all moves when you do the towers of Hanoi with
n disks on the first pole. The number of moves is exactly 2**n. Here is what `hanoi` is supposed to do.

In [ ]:
def test_hanoi():
    protocol = hanoi(2)
    expected = [([2, 1], [], []),    # initial state
                ([2], [1], []),      # move disk 1 from a to b
                ([], [1], [2]),      # move disk 2 from a to c
                ([], [], [2, 1])]    # move disk 1 from b to c
    test_eq(expected, protocol)
    n = 20
    test_eq(len(hanoi(n)), 2**n)

Idea:

(1) A natural representation of the Hanoi towers ar three stacks, named a, b, c.
The first one, a, is initialized to [n, n-1, ..., 1], b and c are empty.

(2) Let `move(k, x, y, z)` denote the move of k disks from stack x to stack z using y as buffer.

The algorithm is recursive:
`move(1, a, b, c)` is obvious: Just perform the move, let the buffer alone.
`move(2, a, b, c)` requires three moves: `(1, a, c, b), (1, a, b, c), (1, b, a, c)`
In exactly the same way the general case `move(n, a, b, c)` can be reduced to three
moves with n-1 instead of n.

In [ ]:
#collapse
def hanoi(n: int) -> list:
    """
    Towers of Hanoi. Exponential time (n = 22 -> 12s)
    :param n: number of disks on first tower > 0
    :return: protocol of all moves
    """

    a, b, c = list(range(n, 0, -1)), [], []     # a = [n, n-1, ..., 1]
    protocol = [(list(a), list(b), list(c))]

    def move(k: int, x: list, y: list, z: list) -> None:
        """
        This function moves k disks from x to z using y
        :param k: number of disks to move
        :param x: stack to move from
        :param y: buffer
        :param z: stack to move to
        """
        if k == 1:
            z.append(x.pop())
            protocol.append((list(a), list(b), list(c)))
        else:
            move(k - 1, x, z, y)
            move(1, x, y, z)
            move(k - 1, y, x, z)

    move(n, a, b, c)
    return protocol

In [ ]:
print(f"{hanoi.__name__ :<6}| {timeit(test_hanoi, number=1)}")

hanoi | 0.7914820000000002
